In [12]:
# notebooks/02_baseline_failure_scenarios.ipynb

import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json

from src.utils.db import SupplyChainDB

# Setup
db = SupplyChainDB()

with open('data/processed/baseline_metrics.json', 'r') as f:
    baseline = json.load(f)

print("="*70)
print("BASELINE SYSTEM FAILURE SCENARIOS")
print("="*70)
print("\nSimple Reorder Point Logic:")
print("  IF inventory <= reorder_point AND no_pending_orders THEN:")
print("      order_quantity = base_demand * supply_days_target * 1.2")
print("      place_order(quantity)")
print("\nProblems: No forecasting, no event awareness, no optimization")
print("="*70)

# %% [markdown]
# # SCENARIO 1: Black Friday Disaster
# 
# **Problem**: Baseline system doesn't anticipate demand spikes
# 
# Black Friday demand is 2.36x normal, but baseline orders normal quantities

# %% Analyze Black Friday period
black_friday_2023 = '2023-11-24'
black_friday_2024 = '2024-11-29'

# Focus on 2023 (we have complete data)
bf_start = pd.to_datetime(black_friday_2023)
bf_end = bf_start + pd.Timedelta(days=4)

# Get sales during Black Friday
bf_sales = db.query(f"""
    SELECT 
        date,
        product_id,
        SUM(units_sold) as units,
        SUM(revenue) as revenue
    FROM sales
    WHERE date >= '{bf_start.date()}' AND date <= '{bf_end.date()}'
    GROUP BY date, product_id
""")

# Get baseline (2 weeks before)
baseline_start = bf_start - pd.Timedelta(days=14)
baseline_end = bf_start - pd.Timedelta(days=1)

baseline_sales = db.query(f"""
    SELECT 
        product_id,
        AVG(units) as avg_units_per_day,
        AVG(revenue) as avg_revenue_per_day
    FROM (
        SELECT 
            date,
            product_id,
            SUM(units_sold) as units,
            SUM(revenue) as revenue
        FROM sales
        WHERE date >= '{baseline_start.date()}' AND date <= '{baseline_end.date()}'
        GROUP BY date, product_id
    )
    GROUP BY product_id
""")

# Compare actual BF demand vs baseline
bf_by_product = bf_sales.groupby('product_id').agg({
    'units': 'sum',
    'revenue': 'sum'
}).reset_index().rename(columns={'units': 'units_bf', 'revenue': 'revenue_bf'})

bf_comparison = bf_by_product.merge(baseline_sales, on='product_id', suffixes=('_bf', '_baseline'))
bf_comparison['demand_multiplier'] = bf_comparison['units_bf'] / (bf_comparison['avg_units_per_day'] * 4)  # 4 days of BF
bf_comparison = bf_comparison.sort_values('demand_multiplier', ascending=False)

print("\n🔥 BLACK FRIDAY 2023 - Top 10 Demand Spikes:")
print(bf_comparison[['product_id', 'demand_multiplier', 'revenue_bf']].head(10).to_string(index=False))

# Get inventory levels during Black Friday
bf_inventory = db.query(f"""
    SELECT 
        date,
        product_id,
        SUM(units_on_hand) as total_inventory,
        SUM(stockout) as stockouts
    FROM inventory_snapshots
    WHERE date >= '{bf_start.date()}' AND date <= '{bf_end.date()}'
    GROUP BY date, product_id
""")

# Find products that stocked out during BF
bf_stockouts = bf_inventory[bf_inventory['stockouts'] > 0]['product_id'].unique()

# Calculate lost revenue (products that stocked out during BF spike)
bf_lost = bf_comparison[bf_comparison['product_id'].isin(bf_stockouts)].copy()

# Estimate lost sales: assume we could have sold baseline_multiplier * baseline if we had stock
bf_lost['potential_additional_revenue'] = (
    bf_lost['avg_revenue_per_day'] * 4 * bf_lost['demand_multiplier'] - bf_lost['revenue_bf']
)

total_bf_lost = bf_lost['potential_additional_revenue'].sum()

print(f"\n💰 BLACK FRIDAY LOST REVENUE:")
print(f"   Products that stocked out: {len(bf_stockouts)}")
print(f"   Estimated lost revenue: ${total_bf_lost:,.2f}")

# Visualize
top_bf_lost = bf_lost.nlargest(10, 'potential_additional_revenue')

fig = go.Figure()

fig.add_trace(go.Bar(
    x=top_bf_lost['product_id'],
    y=top_bf_lost['revenue_bf'],
    name='Actual Revenue',
    marker_color='lightblue'
))

fig.add_trace(go.Bar(
    x=top_bf_lost['product_id'],
    y=top_bf_lost['potential_additional_revenue'],
    name='Lost Revenue (Stockout)',
    marker_color='red'
))

fig.update_layout(
    title='Black Friday 2023: Actual vs Lost Revenue',
    xaxis_title='Product',
    yaxis_title='Revenue ($)',
    barmode='stack',
    height=500
)

fig.show()

# SCENARIO 1 SUMMARY
scenario_1 = {
    'name': 'Black Friday Disaster',
    'lost_revenue': float(total_bf_lost),
    'products_affected': len(bf_stockouts),
    'avg_demand_spike': float(bf_comparison['demand_multiplier'].mean()),
    'description': 'Baseline system orders normal quantities, misses 2.36x demand spike'
}

print(f"\n✅ SCENARIO 1: ${scenario_1['lost_revenue']:,.2f} lost")

✅ Connected to database: data/processed/supply_chain.duckdb
BASELINE SYSTEM FAILURE SCENARIOS

Simple Reorder Point Logic:
  IF inventory <= reorder_point AND no_pending_orders THEN:
      order_quantity = base_demand * supply_days_target * 1.2
      place_order(quantity)

Problems: No forecasting, no event awareness, no optimization

🔥 BLACK FRIDAY 2023 - Top 10 Demand Spikes:
product_id  demand_multiplier  revenue_bf
   TAB-013           4.596154    78203.19
   TAB-014           3.683246    66141.06
   MON-017           3.620690    66974.40
   LAP-005           3.594595    23268.16
   LAP-008           3.500000    46569.60
   MON-016           3.468468    60343.80
   LAP-006           3.410256    25898.90
   TAB-012           3.387097   100188.00
   LAP-003           3.310811    22046.85
   MIC-030           3.269512    13037.32

💰 BLACK FRIDAY LOST REVENUE:
   Products that stocked out: 11
   Estimated lost revenue: $0.00



✅ SCENARIO 1: $0.00 lost


In [13]:
# %% [markdown]
# # SCENARIO 2: Supply Disruption Blindness
# 
# **Problem**: Baseline doesn't react to external events (port delays, factory shutdowns)
# 
# March 2024 laptop supply disruption caused 21 days of reduced supply

# %% Analyze supply disruption
disruption_start = pd.to_datetime('2024-03-10')
disruption_end = disruption_start + pd.Timedelta(days=21)

# Get laptop sales during disruption
disruption_sales = db.query(f"""
    SELECT 
        s.date,
        s.product_id,
        p.category,
        SUM(s.units_sold) as units,
        SUM(s.revenue) as revenue
    FROM sales s
    JOIN products p ON s.product_id = p.product_id
    WHERE p.category = 'Laptops'
        AND s.date >= '{disruption_start.date()}'
        AND s.date <= '{disruption_end.date()}'
    GROUP BY s.date, s.product_id, p.category
""")

# Baseline: 3 weeks before disruption
pre_disruption_start = disruption_start - pd.Timedelta(days=21)
pre_disruption_end = disruption_start - pd.Timedelta(days=1)

pre_disruption_sales = db.query(f"""
    SELECT 
        product_id,
        AVG(units) as avg_units_per_day,
        AVG(revenue) as avg_revenue_per_day
    FROM (
        SELECT 
            s.date,
            s.product_id,
            SUM(s.units_sold) as units,
            SUM(s.revenue) as revenue
        FROM sales s
        JOIN products p ON s.product_id = p.product_id
        WHERE p.category = 'Laptops'
            AND s.date >= '{pre_disruption_start.date()}'
            AND s.date <= '{pre_disruption_end.date()}'
        GROUP BY s.date, s.product_id
    )
    GROUP BY product_id
""")

# Get stockouts during disruption
disruption_stockouts = db.query(f"""
    SELECT 
        i.product_id,
        p.name,
        SUM(i.stockout) as stockout_days,
        p.unit_price,
        p.base_demand_daily
    FROM inventory_snapshots i
    JOIN products p ON i.product_id = p.product_id
    WHERE p.category = 'Laptops'
        AND i.date >= '{disruption_start.date()}'
        AND i.date <= '{disruption_end.date()}'
    GROUP BY i.product_id, p.name, p.unit_price, p.base_demand_daily
    HAVING SUM(i.stockout) > 0
""")

# Calculate lost revenue
disruption_stockouts['lost_revenue'] = (
    disruption_stockouts['stockout_days'] *
    disruption_stockouts['base_demand_daily'] *
    disruption_stockouts['unit_price']
)

total_disruption_lost = disruption_stockouts['lost_revenue'].sum()

print("\n🚨 SUPPLY DISRUPTION (Laptops - March 2024):")
print(f"   Duration: 21 days")
print(f"   Products with stockouts: {len(disruption_stockouts)}")
print(f"   Total stockout days: {disruption_stockouts['stockout_days'].sum():.0f}")
print(f"   Lost revenue: ${total_disruption_lost:,.2f}")

print("\n   Top affected products:")
print(disruption_stockouts.nlargest(5, 'lost_revenue')[['product_id', 'name', 'stockout_days', 'lost_revenue']].to_string(index=False))

# SCENARIO 2 SUMMARY
scenario_2 = {
    'name': 'Supply Disruption Blindness',
    'lost_revenue': float(total_disruption_lost),
    'products_affected': len(disruption_stockouts),
    'duration_days': 21,
    'description': 'System doesnt anticipate port delays, causes extended stockouts'
}

print(f"\n✅ SCENARIO 2: ${scenario_2['lost_revenue']:,.2f} lost")


🚨 SUPPLY DISRUPTION (Laptops - March 2024):
   Duration: 21 days
   Products with stockouts: 0
   Total stockout days: 0
   Lost revenue: $0.00

   Top affected products:
Empty DataFrame
Columns: [product_id, name, stockout_days, lost_revenue]
Index: []

✅ SCENARIO 2: $0.00 lost


In [14]:
# %% [markdown]
# # SCENARIO 3: MOQ Optimization Failure
# 
# **Problem**: Baseline orders just enough to hit reorder point, misses volume discounts
# 
# Missing volume discounts by ordering 199 units instead of 200 (threshold)

# %% Analyze near-miss volume discounts
# Get all orders that were CLOSE to discount threshold but didn't hit it
near_miss_orders = db.query("""
    SELECT 
        po.order_id,
        po.product_id,
        po.supplier_id,
        po.units_ordered,
        po.volume_discount_applied,
        po.total_cost,
        s.volume_discount_threshold_units as threshold,
        s.volume_discount_pct as discount_rate,
        CAST(po.units_ordered AS FLOAT) / s.volume_discount_threshold_units as threshold_ratio
    FROM purchase_orders po
    JOIN suppliers s ON po.supplier_id = s.supplier_id
    WHERE po.volume_discount_applied = FALSE
        AND po.units_ordered >= s.volume_discount_threshold_units * 0.8
        AND po.units_ordered < s.volume_discount_threshold_units
    ORDER BY po.total_cost DESC
""")

# Calculate potential savings if we'd ordered to threshold
near_miss_orders['units_to_threshold'] = (
    near_miss_orders['threshold'] - near_miss_orders['units_ordered']
)

near_miss_orders['potential_savings'] = (
    near_miss_orders['total_cost'] * near_miss_orders['discount_rate']
)

total_moq_opportunity = near_miss_orders['potential_savings'].sum()

print("\n💸 VOLUME DISCOUNT NEAR-MISSES:")
print(f"   Orders within 20% of threshold: {len(near_miss_orders)}")
print(f"   Potential savings if optimized: ${total_moq_opportunity:,.2f}")

print("\n   Top 10 missed opportunities:")
print(near_miss_orders.nlargest(10, 'potential_savings')[
    ['order_id', 'product_id', 'units_ordered', 'threshold', 'potential_savings']
].to_string(index=False))

# Visualize
fig = px.histogram(
    near_miss_orders,
    x='threshold_ratio',
    nbins=20,
    title='Distribution of Orders Near Volume Discount Threshold',
    labels={'threshold_ratio': 'Ratio to Threshold (1.0 = threshold)', 'count': 'Number of Orders'}
)

fig.add_vline(x=1.0, line_dash="dash", line_color="red", annotation_text="Discount Threshold")
fig.show()

# SCENARIO 3 SUMMARY
scenario_3 = {
    'name': 'MOQ Optimization Failure',
    'lost_savings': float(total_moq_opportunity),
    'orders_affected': len(near_miss_orders),
    'avg_savings_per_order': float(near_miss_orders['potential_savings'].mean()),
    'description': 'Orders 80-99% of discount threshold, misses bulk savings'
}

print(f"\n✅ SCENARIO 3: ${scenario_3['lost_savings']:,.2f} opportunity")


💸 VOLUME DISCOUNT NEAR-MISSES:
   Orders within 20% of threshold: 183
   Potential savings if optimized: $296,120.37

   Top 10 missed opportunities:
order_id product_id  units_ordered  threshold  potential_savings
PO-00129    TAB-012            250        300           2597.475
PO-00134    TAB-012            250        300           2597.475
PO-00148    TAB-012            250        300           2597.475
PO-00165    TAB-012            250        300           2597.475
PO-00169    TAB-012            250        300           2597.475
PO-00222    TAB-012            250        300           2597.475
PO-00254    TAB-012            250        300           2597.475
PO-00281    TAB-012            250        300           2597.475
PO-00310    TAB-012            250        300           2597.475
PO-00323    TAB-012            250        300           2597.475



✅ SCENARIO 3: $296,120.37 opportunity


In [15]:
# %% [markdown]
# # SCENARIO 4: Multi-Product Consolidation Waste
# 
# **Problem**: Baseline orders products separately even from same supplier
# 
# Multiple orders to same supplier on same day = wasted shipping costs

# %% Find same-day, same-supplier orders that could be consolidated
same_supplier_same_day = db.query("""
    SELECT 
        order_date,
        supplier_id,
        COUNT(*) as num_orders,
        SUM(shipping_cost) as total_shipping,
        SUM(total_cost) as total_cost,
        SUM(units_ordered) as total_units
    FROM purchase_orders
    GROUP BY order_date, supplier_id
    HAVING COUNT(*) > 1
    ORDER BY total_shipping DESC
""")

# Estimate savings: if consolidated, would pay shipping once instead of multiple times
# Assume consolidation saves all but largest shipping cost
consolidation_details = db.query("""
    WITH ranked_orders AS (
        SELECT 
            order_date,
            supplier_id,
            order_id,
            shipping_cost,
            ROW_NUMBER() OVER (PARTITION BY order_date, supplier_id ORDER BY shipping_cost DESC) as rn
        FROM purchase_orders
    )
    SELECT 
        order_date,
        supplier_id,
        SUM(shipping_cost) as total_shipping,
        SUM(CASE WHEN rn > 1 THEN shipping_cost ELSE 0 END) as wasted_shipping
    FROM ranked_orders
    GROUP BY order_date, supplier_id
    HAVING COUNT(*) > 1
""")

total_consolidation_savings = consolidation_details['wasted_shipping'].sum()

print("\n📦 CONSOLIDATION OPPORTUNITIES:")
print(f"   Days with multiple orders to same supplier: {len(same_supplier_same_day)}")
print(f"   Total wasted shipping costs: ${total_consolidation_savings:,.2f}")

print("\n   Top 10 days by wasted shipping:")
print(consolidation_details.nlargest(10, 'wasted_shipping')[
    ['order_date', 'supplier_id', 'total_shipping', 'wasted_shipping']
].to_string(index=False))

# SCENARIO 4 SUMMARY
scenario_4 = {
    'name': 'Multi-Product Consolidation Waste',
    'wasted_shipping': float(total_consolidation_savings),
    'opportunities': len(same_supplier_same_day),
    'avg_waste_per_occurrence': float(consolidation_details['wasted_shipping'].mean()),
    'description': 'Separate orders to same supplier on same day = extra shipping'
}

print(f"\n✅ SCENARIO 4: ${scenario_4['wasted_shipping']:,.2f} wasted")


📦 CONSOLIDATION OPPORTUNITIES:
   Days with multiple orders to same supplier: 226
   Total wasted shipping costs: $300,607.63

   Top 10 days by wasted shipping:
order_date supplier_id  total_shipping  wasted_shipping
2023-11-24       SUP-A        22243.50         16710.00
2023-01-11       SUP-C        13788.00          9108.00
2023-01-09       SUP-B        11069.19          8416.44
2023-01-17       SUP-B         9879.30          7591.05
2023-01-01       SUP-C        11203.20          6775.20
2024-08-14       SUP-A        11913.00          6379.50
2024-11-29       SUP-A        11888.50          6355.00
2023-11-08       SUP-A        11843.00          6309.50
2024-09-26       SUP-A        11205.00          6154.50
2024-03-15       SUP-C         9993.60          5565.60

✅ SCENARIO 4: $300,607.63 wasted


In [16]:
# %% [markdown]
# # SCENARIO 5: Supplier Selection Inefficiency
# 
# **Problem**: Baseline uses assigned supplier without considering alternatives
# 
# Sometimes cheaper supplier available with acceptable lead time trade-off

# %% Analyze supplier selection opportunities
# Get all orders and compare to alternative suppliers
orders_with_alternatives = db.query("""
    WITH order_details AS (
        SELECT 
            po.order_id,
            po.product_id,
            po.supplier_id,
            po.units_ordered,
            po.total_cost,
            po.unit_cost,
            po.lead_time_actual,
            p.category
        FROM purchase_orders po
        JOIN products p ON po.product_id = p.product_id
    )
    SELECT 
        od.*,
        s.supplier_id as alt_supplier_id,
        s.lead_time_days as alt_lead_time,
        p.unit_cost as product_base_cost
    FROM order_details od
    CROSS JOIN suppliers s
    JOIN products p ON od.product_id = p.product_id
    WHERE s.supplier_id != od.supplier_id
""")

# For each order, calculate what it would have cost with alternative supplier
# Simplified: assume unit_cost varies by ±10% between suppliers
np.random.seed(42)
orders_with_alternatives['alt_unit_cost'] = (
    orders_with_alternatives['unit_cost'] * 
    np.random.uniform(0.9, 1.1, len(orders_with_alternatives))
)

orders_with_alternatives['alt_total_cost'] = (
    orders_with_alternatives['alt_unit_cost'] * 
    orders_with_alternatives['units_ordered']
)

orders_with_alternatives['potential_savings'] = (
    orders_with_alternatives['total_cost'] - 
    orders_with_alternatives['alt_total_cost']
)

# Filter for cases where alternative would save >5% and lead time difference <7 days
significant_alternatives = orders_with_alternatives[
    (orders_with_alternatives['potential_savings'] > orders_with_alternatives['total_cost'] * 0.05) &
    (abs(orders_with_alternatives['alt_lead_time'] - orders_with_alternatives['lead_time_actual']) <= 7)
]

total_supplier_optimization = significant_alternatives.groupby('order_id')['potential_savings'].max().sum()

print("\n🔄 SUPPLIER SELECTION OPTIMIZATION:")
print(f"   Orders with better alternative: {significant_alternatives['order_id'].nunique()}")
print(f"   Potential savings: ${total_supplier_optimization:,.2f}")

# SCENARIO 5 SUMMARY
scenario_5 = {
    'name': 'Supplier Selection Inefficiency',
    'potential_savings': float(total_supplier_optimization),
    'orders_affected': int(significant_alternatives['order_id'].nunique()),
    'description': 'Fixed supplier assignment ignores cost-effective alternatives'
}

print(f"\n✅ SCENARIO 5: ${scenario_5['potential_savings']:,.2f} opportunity")


🔄 SUPPLIER SELECTION OPTIMIZATION:
   Orders with better alternative: 228
   Potential savings: $676,181.09

✅ SCENARIO 5: $676,181.09 opportunity


In [18]:
# %% [markdown]
# # TOTAL OPPORTUNITY ANALYSIS

# %% Aggregate all scenarios
scenarios = [scenario_1, scenario_2, scenario_3, scenario_4, scenario_5]

total_opportunity = sum([
    scenario_1['lost_revenue'],
    scenario_2['lost_revenue'],
    scenario_3['lost_savings'],
    scenario_4['wasted_shipping'],
    scenario_5['potential_savings']
])

print("\n" + "="*70)
print("BASELINE SYSTEM FAILURE ANALYSIS - SUMMARY")
print("="*70)

for i, scenario in enumerate(scenarios, 1):
    amount = scenario.get('lost_revenue', scenario.get('lost_savings', scenario.get('wasted_shipping', scenario.get('potential_savings'))))
    pct_of_total = (amount / total_opportunity) * 100
    print(f"\n{i}. {scenario['name']}:")
    print(f"   Impact: ${amount:,.2f} ({pct_of_total:.1f}% of total)")
    print(f"   {scenario['description']}")

print("\n" + "="*70)
print(f"TOTAL OPPORTUNITY: ${total_opportunity:,.2f}")
print("="*70)

print(f"\nAs % of total procurement (${baseline['total_procurement']:,.2f}):")
print(f"  Potential improvement: {(total_opportunity / baseline['total_procurement']) * 100:.1f}%")

print(f"\nAs % of total revenue (${baseline['total_revenue']:,.2f}):")
print(f"  Potential margin improvement: {(total_opportunity / baseline['total_revenue']) * 100:.2f}%")

# Save scenarios to JSON
import json
scenarios_summary = {
    'scenarios': scenarios,
    'total_opportunity': total_opportunity,
    'baseline_procurement': baseline['total_procurement'],
    'baseline_revenue': baseline['total_revenue'],
    'improvement_potential_pct': (total_opportunity / baseline['total_procurement']) * 100
}

with open('data/processed/failure_scenarios.json', 'w') as f:
    json.dump(scenarios_summary, f, indent=2)

print("\n✅ Scenarios saved to: data/processed/failure_scenarios.json")


BASELINE SYSTEM FAILURE ANALYSIS - SUMMARY

1. Black Friday Disaster:
   Impact: $0.00 (0.0% of total)
   Baseline system orders normal quantities, misses 2.36x demand spike

2. Supply Disruption Blindness:
   Impact: $0.00 (0.0% of total)
   System doesnt anticipate port delays, causes extended stockouts

3. MOQ Optimization Failure:
   Impact: $296,120.37 (23.3% of total)
   Orders 80-99% of discount threshold, misses bulk savings

4. Multi-Product Consolidation Waste:
   Impact: $300,607.63 (23.6% of total)
   Separate orders to same supplier on same day = extra shipping

5. Supplier Selection Inefficiency:
   Impact: $676,181.09 (53.1% of total)
   Fixed supplier assignment ignores cost-effective alternatives

TOTAL OPPORTUNITY: $1,272,909.09

As % of total procurement ($43,311,028.03):
  Potential improvement: 2.9%

As % of total revenue ($78,344,402.27):
  Potential margin improvement: 1.62%

✅ Scenarios saved to: data/processed/failure_scenarios.json


In [19]:
# %% Visualization: Opportunity breakdown
scenario_df = pd.DataFrame([
    {
        'scenario': s['name'],
        'amount': s.get('lost_revenue', s.get('lost_savings', s.get('wasted_shipping', s.get('potential_savings')))),
        'type': 'Lost Revenue' if 'lost_revenue' in s else 'Cost Savings'
    }
    for s in scenarios
])

fig = px.bar(
    scenario_df,
    x='scenario',
    y='amount',
    color='type',
    title=f'Baseline System Failures - Total Opportunity: ${total_opportunity:,.0f}',
    labels={'amount': 'Opportunity ($)', 'scenario': 'Failure Scenario'},
    text='amount',
    color_discrete_map={'Lost Revenue': 'red', 'Cost Savings': 'orange'}
)

fig.update_traces(texttemplate='$%{text:,.0f}', textposition='outside')
fig.update_layout(xaxis_tickangle=-45, height=600)
fig.show()

print("\n" + "="*70)
print("NEXT STEPS:")
print("="*70)
print("These failure scenarios will be our test cases for the agentic system.")
print("Goal: Reduce total opportunity from ${:,.2f} to <${:,.2f}".format(
    total_opportunity, total_opportunity * 0.3
))
print("="*70)


NEXT STEPS:
These failure scenarios will be our test cases for the agentic system.
Goal: Reduce total opportunity from $1,272,909.09 to <$381,872.73
